In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]


In [2]:
startTm = datetime.datetime.now()

startDate = '20201214'
endDate = '20201216'

readPath = '/mnt/equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

for thisDate in dateLs:
    
    readPath = '/mnt/equityTradeLogs'
    rawOrderLog = pd.read_csv(os.path.join(readPath, 'speedCompare_%s.csv'%thisDate))
    for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
                'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
        rawOrderLog[col] = rawOrderLog[col].fillna(0)
        rawOrderLog[col] = rawOrderLog[col].astype('int64')   
    rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

    rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]
    
    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep=False)]\
    .groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep='first')
    
    display('There are ticks with orderDirection 0')
    display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
                'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

    assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
    try:
        assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
    except:
        print('There are orders with all things same except sdd')
        print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId'], keep=False)])
        assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
    try:
        assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
    except:
        print('There are orders with same internalId but different orderId other than accCode 8856 case')
        print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

    r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
    r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
    r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize']).grouper.group_info[0]
    r1 = r1.sort_values(by=['test', 'clockAtArrival'])
    r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
    r1['vai'] = r1.groupby('test')['vai'].ffill()
    r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
    r2 = r2.sort_values(by=['test', 'clockAtArrival'])
    r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
    r2['vai'] = r2.groupby('test')['vai'].ffill()
    assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
    try:
        assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
    except:
        a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
        print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
    rawOrderLog = pd.concat([r1, r2])
    del r1
    del r2  
    
    rawOrderLog = rawOrderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
    rawOrderLog['clock'] = rawOrderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    rawOrderLog["broker"] = np.where(rawOrderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), rawOrderLog['accCode'] // 10000, rawOrderLog['accCode'] // 100)
    rawOrderLog['colo_broker'] = rawOrderLog['colo'].str[:2] + '_' + rawOrderLog['broker'].astype('str')
    rawOrderLog['colo_account'] = rawOrderLog['colo'].str[:2] + '_' + rawOrderLog['accCode'].astype('str')
    rawOrderLog['order'] = rawOrderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'internalId']).grouper.group_info[0]
    rawOrderLog['group'] = rawOrderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
    rawOrderLog['startClock'] = rawOrderLog.groupby(['order'])['clockAtArrival'].transform('first')
    rawOrderLog['duration'] = rawOrderLog['clockAtArrival'] - rawOrderLog['startClock']
    rawOrderLog['orderPrice'] = rawOrderLog['orderPrice'].apply(lambda x: round(x, 2))
    rawOrderLog['tradePrice'] = rawOrderLog['tradePrice'].apply(lambda x: round(x, 2))
    rawOrderLog['orderDirection1'] = np.where(rawOrderLog["orderDirection"] == -2, -1, np.where(
        rawOrderLog["orderDirection"] == 2, 1, rawOrderLog["orderDirection"]))
    orderLog = rawOrderLog.copy()
    
    
    ### Assertion 1:  make sure same direction in same date, secid, vai
    print('=======================================================================================')
    print('1. same date, secid, vai: same direction')
    orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
    if len(orderLog[orderLog['directNum'] != 1]) > 0:
        print('opposite direction for same date, same secid, same vai')
        display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0)][['date', 'accCode', 'secid', 'vai', 'orderDirection', 'order']])
        orderLog = orderLog[orderLog['directNum'] == 1]

    assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

    ## Assertion 2:  make sure each account, secid, vai only has one insertion
    print('=======================================================================================')
    print('2. same date, secid, vai, accCode: one insertion')
    a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
    if len(a[a > 1]) > 0:
        print('more than one insertion at same time')
        a = a[a>1].reset_index()
        display(a)
        orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

    orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                                 np.where(orderLog['mse'] == 100, 1, 0), np.nan)
    orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

    placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
    print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


    ### Assertion 3:  check IPO stocks selling status
    print('=======================================================================================')
    print('3. IPO stocks selling (ars = 301, 302)')
    if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
        kk = orderLog[orderLog['ars'].isin([301, 302])]
        print(kk)
        try:
            assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
            print('we only sell, never buy')
        except:
            print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            print(kk[kk['orderDirection1'] == 1])
        kk1 = kk[kk['updateType'] == 0]
        kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
        kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
        kk1['diff'] = kk1['diff'].fillna(0)
        try:
            assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
            print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
        except:
            print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            print(kk1[kk1['diff'] < 10e6])
        kk2 = kk[(kk['updateType'] == 1)]
        try:
            assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
            print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
        except:
            print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            print(kk2[kk2['duration'] < 3e6])


    ### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
    print('=======================================================================================')
    print('4. updateType 7 orders')
    if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
        assert(orderLog[orderLog['updateType'] == 7].groupby('accCode')['order'].nunique().max() < 20)
        assert(orderLog[orderLog['updateType'] == 7].groupby('accCode')['order'].nunique().sum() < 100)

    ### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
    print('=======================================================================================')
    print('5. updateType 6 orders')
    k1 = orderLog[orderLog['updateType'] == 6].groupby('accCode')['order'].nunique().reset_index()
    k2 = orderLog.groupby('accCode')['order'].nunique().reset_index()
    k = pd.merge(k1, k2, on='accCode', how='left')
    k['prob'] = k['order_x']/k['order_y']
    try:
        assert(sum(k['prob'] >= 0.05) == 0)
    except:
        print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(k[k['prob'] >= 0.05])

    ### Assertion 6: check CYB orders, make sure CYB stocks total absOrderSize < 30w
    print('=======================================================================================')
    print('6. CYB stocks total order size < 30w')
    try:
        assert(orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)]['absOrderSize'].max() <= 300000)
    except:
        print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')


    ### Assertion 7:  make sure there is no unexpected updateType 
    print('=======================================================================================')
    print('7. unexpected updateType')
    def getTuple(x):
        return tuple(i for i in x)

    checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
    checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
    checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                         np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                                   (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                                   (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                         np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))

    orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
    orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

    ### Assertion 8:  make sure status==0 got all traded
    print('=======================================================================================')
    print('8. status == 0: all traded')
    a = orderLog[orderLog['status'] == 0]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    print('in total trade, any fill != total cases')
    display(a[a['filled'] != a['total']])
    if a[a['filled'] != a['total']].shape[0] > 0:
        removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### Assertion 9:  make sure status==1 got partial traded
    print('=======================================================================================')
    print('9. status == 1: partial traded')
    a = orderLog[orderLog['status'] == 1]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
    display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
    if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
        removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### Assertion 10: make sure no cancellation within 1 sec
    print('=======================================================================================')
    print('10. no cancellation within 1 sec')
    a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
    print('any cancellation within 1 sec')
    display(a)
    if a.shape[0] > 0:
        removeOrderLs = a['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


    ### Assertion 11: make sure no order has shares > 80w or notional > 800w
    print('=======================================================================================')
    print('11. Orders with size > 80w or notional > 800w')
    orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
    if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
        print('some order quantity are > 80w')
        print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
        display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                             'orderNtl', 'orderDirection', 'clock', 'order']])

    if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
        print('some order ntl are > 800w')
        print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
        display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                          'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                          "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

    removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
    orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
    orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
    orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
    orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')

    orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
    orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
    if orderLog[orderLog['mrsb90'] == '-'].shape[0] != 0:
        display(orderLog[orderLog['mrsb90'] == '-'])
    orderLog = orderLog[orderLog['mrsb90'] != '-']
    orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
    if orderLog[orderLog['aaa'] == '-'].shape[0] != 0:
        display(orderLog[orderLog['aaa'] == '-'])
    orderLog = orderLog[orderLog['aaa'] != '-']
    orderLog['aaa'] = orderLog['aaa'].astype(float)
    orderLog.loc[(orderLog['orderDirection'] >= 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
    orderLog.loc[(orderLog['orderDirection'] >= 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

    orderLog.loc[(orderLog['orderDirection'] >= 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
    orderLog.loc[(orderLog['orderDirection'] >= 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

    orderLog.loc[(orderLog['orderDirection'] < 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
    orderLog.loc[(orderLog['orderDirection'] < 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

    orderLog.loc[(orderLog['orderDirection'] < 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
    orderLog.loc[(orderLog['orderDirection'] < 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']  

    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)


    orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
    orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
    orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
    orderLog = orderLog[orderLog['secid'] >= 2000000].reset_index(drop=True)
    print('start to load data')    
    readPath = '/mnt/orderLog/mdData'
    rawMsgDataSZ = pd.read_pickle(os.path.join(readPath, 'mdLog_msg_%s.pkl'%thisDate))
    print('load data finished')
    orderDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '2'][['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'OrderQty', 'Price', 'cum_volume', "TransactTime"]].reset_index(drop=True)
    orderDataSZ['updateType'] = 0
    tradeDataSZ = pd.concat([rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'BidApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]],
                             rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]], sort=False)
    tradeDataSZ['ApplSeqNum'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), tradeDataSZ['OfferApplSeqNum'], tradeDataSZ['BidApplSeqNum'])
    tradeDataSZ['Side'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), 2, 1)
    tradeDataSZ = tradeDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]
    tradeDataSZ['updateType'] = 4
    cancelDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '4'][['SecurityID', 'BidApplSeqNum', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]].reset_index(drop=True)
    cancelDataSZ['ApplSeqNum'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, cancelDataSZ['OfferApplSeqNum'], cancelDataSZ['BidApplSeqNum'])
    cancelDataSZ['Side'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, 2, 1)
    cancelDataSZ = cancelDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradeQty', 'cum_volume', "TransactTime"]]
    cancelDataSZ['updateType'] = 3

    msgDataSZ = pd.concat([orderDataSZ, tradeDataSZ, cancelDataSZ], sort=False)
    del orderDataSZ
    del tradeDataSZ
    del cancelDataSZ
    msgDataSZ = msgDataSZ.sort_values(by=['SecurityID', 'ApplSeqNum', 'sequenceNo']).reset_index(drop=True)
    
    msgDataSZ['TradePrice'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradePrice'], 0)
    msgDataSZ['TradePrice'] = msgDataSZ['TradePrice'].astype('int64')
    msgDataSZ['TradeQty'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradeQty'], 0)
    msgDataSZ['TradeQty'] = msgDataSZ['TradeQty'].astype('int64')
    msgDataSZ['secid'] = msgDataSZ['SecurityID'] + 2000000
    assert(msgDataSZ['ApplSeqNum'].max() < 1e8)
    msgDataSZ['StockSeqNum'] = msgDataSZ['SecurityID']*1e8 + msgDataSZ['ApplSeqNum']
    msgDataSZ['date'] = int(thisDate)
    msgDataSZ['startVolume'] = msgDataSZ.groupby(['StockSeqNum'])['cum_volume'].transform('first')
            
    ### order insertion position
    startPos = orderLog[(orderLog['date'] == int(thisDate)) & (orderLog['updateType'] == 0) & (orderLog['secid'] >= 2000000) & (orderLog["isMsg"] == 1)]
    # here!!!!!!!!!! drop duplicates
    startPos = startPos.drop_duplicates(subset=['date', 'secid', 'vai'])
    
    startPos = startPos[((startPos.clock.dt.time >= datetime.time(9, 33)) & (startPos.clock.dt.time <= datetime.time(11, 30))) |\
                        ((startPos.clock.dt.time >= datetime.time(13, 3)) & (startPos.clock.dt.time <= datetime.time(14, 55)))]
#     startPos = startPos[startPos.clock.dt.time < datetime.time(9, 33)]
    
    startPos['SecurityID'] = startPos['secid']-2000000
    startPos['orderDirection1'] = np.where(startPos['orderDirection1'] == 1, 1, 2)
    startPos['cum_volume'] = startPos['vai']
    startPos = startPos[['SecurityID', 'cum_volume', 'orderDirection1', 'accCode', 'absOrderSize', 'vai', 'group']]
    startPos['stockGroup'] = startPos.groupby(['accCode', 'SecurityID']).grouper.group_info[0]
    startPos = startPos.sort_values(by=['SecurityID', 'cum_volume']).reset_index(drop=True)

    
    ### generate order status change data
    infoData = orderLog[(orderLog['date'] == int(thisDate)) & (orderLog['group'].isin(startPos['group'].unique())) & (orderLog['updateType'].isin([0, 3, 4]))].reset_index(drop=True)
    infoData['Price'] = infoData['orderPrice'].apply(lambda x: round(x*100, 0))
    infoData['Price'] = infoData['Price'].astype('int64')*100
    infoData['OrderQty'] = infoData['absOrderSize']
    infoData['Side'] = np.where(infoData['orderDirection1'] == 1, 1, 2)
    infoData['TradePrice'] = np.where(infoData['updateType'] == 4, round(infoData['tradePrice']*100, 0), 0)
    infoData['TradePrice'] = infoData['TradePrice'].astype('int64')*100
    statusInfo = infoData.groupby(['order'])['updateType'].apply(lambda x: tuple(x)).reset_index()
    statusInfo.columns = ['order', 'statusLs']
    tradePriceInfo = infoData.groupby(['order'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
    tradePriceInfo.columns = ['order', 'TradePriceLs']
    tradeQtyInfo = infoData.groupby(['order'])['absFilledThisUpdate'].apply(lambda x: tuple(x)).reset_index()
    tradeQtyInfo.columns = ['order', 'TradeQtyLs']
    infoData["abstradeNtl"] = infoData["absFilledThisUpdate"] * infoData["TradePrice"]
    infoData["TradeNtl"] = infoData.groupby(['order'])['abstradeNtl'].transform('sum')
    infoData = infoData[infoData['updateType'] == 0]
    infoData = pd.merge(infoData, statusInfo, how='left', on=['order'], validate='one_to_one')
    infoData = pd.merge(infoData, tradePriceInfo, how='left', on=['order'], validate='one_to_one')
    infoData = pd.merge(infoData, tradeQtyInfo, how='left', on=['order'], validate='one_to_one')
    infoData['brokerNum'] = infoData.groupby(['date', 'secid', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs'])['colo_account'].transform('count')
    infoData['brokerLs'] = infoData.groupby(['date', 'secid', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs'])['colo_account'].transform(lambda x: ','.join(sorted(x.unique())))
    display(infoData[(infoData["brokerNum"] >= 2) & (infoData["orderDirection"].isin([-2, 2]))]["group"].nunique())
    display(infoData[(infoData["brokerNum"] < 2) & (infoData["orderDirection"].isin([-2, 2]))]["group"].nunique())
    display(infoData[(infoData["brokerNum"] >= 2) & (infoData["orderDirection"].isin([-2, 2]))]["group"].unique())
    gl = infoData[(infoData["brokerNum"] >= 2) & (infoData["orderDirection"].isin([-2, 2]))]["group"].unique()
    gl1 = infoData["group"].unique()
    print(len(gl1))
    print(len(startPos["group"].unique()))
    infoData = infoData.drop_duplicates(subset=['date', 'secid', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs']).reset_index(drop=True)
    infoData = infoData[['date', 'secid', 'vai', 'ars', "mrstaat", "mrstauc", 'isMsg', 'TradeNtl', 'exchange', 'group', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'brokerNum', 'brokerLs', 'order']]
    
    ### find all orders in market that inserted with us
    checkLog = msgDataSZ[msgDataSZ['updateType'] == 0][['StockSeqNum', 'SecurityID', 'cum_volume', 'sequenceNo', 'clockAtArrival', 'Side', 'OrderQty', 'Price', "TransactTime"]].reset_index(drop=True)
    checkLog = checkLog.sort_values(by=['SecurityID', 'sequenceNo'])
    checkLog = pd.merge(checkLog, startPos, how='outer', on=['SecurityID', 'cum_volume'], validate='many_to_one')
#     del startPos
    if checkLog[(~checkLog['vai'].isnull()) & (checkLog['clockAtArrival'].isnull())].shape[0] > 0:
        print('some order vai are wrong')
        removeGroup = checkLog[(~checkLog['vai'].isnull()) & (checkLog['clockAtArrival'].isnull())]['stockGroup'].unique()
        print(removeGroup)
        print(len(gl1))
        print(len(set(checkLog["group"].unique())))
        checkLog = checkLog[~checkLog['stockGroup'].isin(removeGroup)].reset_index(drop=True)
        display(len(set(gl1) - set(checkLog["group"].unique())))
    
    assert(checkLog[(~checkLog['vai'].isnull()) & (checkLog['clockAtArrival'].isnull())].shape[0] == 0)
    checkLog['lastClockInVol'] = checkLog.groupby(['SecurityID', 'cum_volume'])['clockAtArrival'].transform('last')
    checkLog['startClock'] = np.where((~checkLog['group'].isnull()) & (checkLog['clockAtArrival'] == checkLog['lastClockInVol']),
                                      checkLog['clockAtArrival'], np.nan)
    checkLog['group'] = checkLog['group'].ffill()
    checkLog['startClock'] = checkLog.groupby(['SecurityID', 'group'])['startClock'].transform('max')
    checkLog['endClock'] = checkLog['startClock'] + 20*1e3
    checkLog = pd.concat([checkLog[checkLog['clockAtArrival'] == checkLog['startClock']], 
                          checkLog[(checkLog['clockAtArrival'] > checkLog['startClock'] + 10) & 
                                   (checkLog['clockAtArrival'] <= checkLog['endClock'])]])
    checkLog = checkLog.sort_values(by=['SecurityID', 'group'])
    checkLog['vai'] = checkLog.groupby(['SecurityID', 'group'])['vai'].ffill()
    checkLog['orderDirection1'] = checkLog.groupby(['SecurityID', 'group'])['orderDirection1'].ffill()
    checkLog = pd.concat([checkLog[checkLog['clockAtArrival'] == checkLog['startClock']],
                         checkLog[(checkLog['Side'] == checkLog['orderDirection1']) & (checkLog['clockAtArrival'] != checkLog['startClock'])]]).sort_values(by=['SecurityID', 'group'])
     
    group_list = checkLog["group"].unique()

    
    checkLog = pd.merge(msgDataSZ, checkLog[['StockSeqNum', 'vai', 'group']], how='left', on=['StockSeqNum'], validate='many_to_one')
    del msgDataSZ
    checkLog = checkLog[~checkLog['group'].isnull()]
    # 上面能不能简化成inner merge!!!!!!!!!!!!!!!!!!!!!!!!!
    statusInfo = checkLog.groupby(['StockSeqNum'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
    statusInfo.columns = ['StockSeqNum', 'statusLs']
    tradePriceInfo = checkLog.groupby(['StockSeqNum'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
    tradePriceInfo.columns = ['StockSeqNum', 'TradePriceLs']
    tradeQtyInfo = checkLog.groupby(['StockSeqNum'])['TradeQty'].apply(lambda x: tuple(x)).reset_index()
    tradeQtyInfo.columns = ['StockSeqNum', 'TradeQtyLs']
    checkLog = checkLog[checkLog['updateType'] == 0]
    checkLog = pd.merge(checkLog, statusInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
    checkLog = pd.merge(checkLog, tradePriceInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
    checkLog = pd.merge(checkLog, tradeQtyInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
    
    infoData = infoData[infoData["group"].isin(group_list)]

    
    checkLog = pd.merge(checkLog, infoData, how='outer', on=['date', 'secid', 'group', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs'], validate='many_to_one')
    display(set(gl) & set(checkLog["group"].unique()))
    display(len(set(gl1) & set(checkLog["group"].unique())))
    checkLog = checkLog.sort_values(by=['date', 'secid', 'vai', 'sequenceNo']).reset_index(drop=True)
    ### orderType 1 orders have 0 order price, replace 0 with group price
    checkLog['groupPrice'] = checkLog.groupby(['group'])['Price'].transform('median')
    checkLog['Price'] = np.where(checkLog['Price'] == 0, checkLog['groupPrice'], checkLog['Price'])
    checkLog['OrderNtl'] = checkLog['Price'] * checkLog['OrderQty'] / 10000
    
    savePath = '/mnt/orderLog/result/marketPos'
    checkLog.reset_index(drop=True).to_pickle(os.path.join(savePath, 'marketPos1_%s.pkl'%thisDate))
    
print(datetime.datetime.now() - startTm)

/home/work516/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (38,39,55,63) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'There are accounts with duplicated ticks:'

Series([], Name: ars, dtype: int64)

'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
106636,20201214,zt_96_09,965801,1601222,-1,7,-1.0,0,0,-1.0,824.0
295137,20201214,zs_94_03,9461,2002044,-1,7,34444.0,0,0,-1.0,-1.0
330239,20201214,zs_64_01,6480,2002293,-1,1,47893.0,0,0,-1.0,103822.0
335644,20201214,zs_52_06,5269,2002329,-1,1,37688.0,0,0,-1.0,696244.0
342409,20201214,zs_52_09,5291,2002390,-1,1,51351.0,0,0,-1.0,1331409.0
346335,20201214,zs_64_01,6480,2002411,-1,1,53411.0,0,0,-1.0,151687.0
354862,20201214,zs_54_01,5474,2002472,-1,1,47572.0,0,0,-1.0,868911827.0
422367,20201214,zs_52_06,5269,2002845,-1,1,39762.0,0,0,-1.0,901829.0
431413,20201214,zs_52_06,5269,2002890,-1,1,37463.0,0,0,-1.0,668323.0
539711,20201214,zs_52_06,5269,2300410,-1,1,35229.0,0,0,-1.0,305297.0


...

69919
69919
some order vai are wrong
[5660.]
69919
69920


1

{32630,
 32632,
 33544,
 33643,
 33665,
 33666,
 33675,
 33680,
 33684,
 33696,
 33697,
 33705,
 33768,
 33770,
 34083,
 34117,
 34118,
 34119,
 34126,
 34133,
 34221,
 34222,
 34223,
 34226,
 34229,
 34230,
 34238,
 34244,
 34401,
 34804,
 35010,
 35274,
 35280,
 35295,
 35300,
 35309,
 35472,
 35487,
 35848,
 35850,
 35852,
 35854,
 35858,
 35862,
 35874,
 35891,
 35896,
 35897,
 36358,
 36360,
 36361,
 36535,
 36540,
 36541,
 36573,
 36574,
 36579,
 36580,
 36581,
 36587,
 36588,
 36589,
 36604,
 37327,
 37328,
 37339,
 37341,
 37605,
 37606,
 37609,
 37613,
 37614,
 37635,
 37643,
 37651,
 37652,
 37657,
 37659,
 37661,
 37666,
 37669,
 37670,
 37674,
 37675,
 37677,
 38191,
 38213,
 38214,
 38254,
 38273,
 38289,
 38537,
 38546,
 38681,
 38682,
 38683,
 38684,
 38786,
 38802,
 39267,
 39268,
 39289,
 39292,
 39312,
 39702,
 39728,
 39733,
 39735,
 39740,
 40055,
 40083,
 40111,
 40122,
 40129,
 40134,
 40135,
 40179,
 40180,
 40191,
 40246,
 40260,
 40463,
 40607,
 40614,
 40615,


69918

0:49:21.956215
